## References
1. https://youtu.be/kYeLBZMTLjk?si=0g3gF3MtRbBiXVFd

In [2]:
import numpy as np
import matplotlib.pyplot as plt
# from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNOrmalization, Dropout
import tensorflow
from tensorflow.keras.optimizers import Adam


In [ ]:
# Height and width for image size
# channels are color channels amount of red, green, blue
image_dimensions = {'height':256,'width':256,'channels':3}

## Create Classifier Class

In [ ]:
class Classifier:
  def _init_():
    self.model = 0

  def predict(self,x):
    return self.model.predict(x)

  def fit(self,x,y):
    return self.model.train_on_batch(x,y)

  def get_accuracy(self,x,y):
    return self.model.test_on_batch(x,y)

  def load(self,path):
    self.model.load_weights(path)

## Create MesoNet class using Classifier

In [ ]:
class Meso4(Classifier):
  def _init_(self, learning_rate=0.001):
    self.model = self.init_model()
    optimizer = Adam(lr = learning_rate)
    self.model.compile(optimzer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])
  def init_model(self):
    x = Input(shape = (image_dimensions['height'], image_dimensions['width'], image_dimensions['channels']))

    x1 = Conv2D(8,(3,3), padding = 'same', activation='relu')(x)
    x1 = BatchNormalization()(x1)

    x2 = Conv2D(8,(5,5), padding = 'same', activation='relu')(x1)
    x2 = BatchNormalization()(x2)
    x2 = MaxPooling2D(pool_size=(2,2), padding = 'same')(x2)

    x3 = Conv2D(16,(5,5), padding = 'same', activation='relu')(x2)
    x3 = BatchNormalization()(x3)
    x3 = MaxPooling2D(pool_size=(4,4), padding = 'same')(x4)

    y = Flatten()(x4)
    y = Dropout(0.5)(y)
    y = Dense(16)(y)
    y= LeakyReLU(alpha=0.1)(y)
    y= Dropout(0.5)(y)
    y = Dense(1, activation='sigmoid')(y)

    return Model(inputs=x, outputs=y)






## Prepare model with pretrained weights

In [ ]:
meso = Meso4()
meso.load('./weights/Meso4_DF')

## Image Processing Pipeline

## Data Generator

> Add blockquote



## Show the amount of images and classes used for training dataset

In [ ]:
# Rescaling pixel values (between 1 and 255) to a range between 0 and 1
dataGenerator = ImageDataGenerator(rescale=1./255)

#Instanting generator to feed images through the network
generator = dataGenerator.flow_from_dictionary(
    './data/',
    target_size=(256,256),
    batch_size = 1,
    class_mode = 'binary'
)


## Check Class Assignment

In [ ]:
generator.class_indices

## Jupyter Specific Code
Account for hidden file for autosaves but needs to be removed for flow from dictionary to work

In [ ]:
#uncomment to use
# !rmdir /s /q c:data\.ipynb_checkpoints

## Making Generator after removing

In [ ]:
# Rescaling pixel values (between 1 and 255) to a range between 0 and 1
dataGenerator = ImageDataGenerator(rescale=1./255)

#Instanting generator to feed images through the network
generator = dataGenerator.flow_from_dictionary(
    './data/',
    target_size=(256,256),
    batch_size = 1,
    class_mode = 'binary'
)

#check class assignment after removal
generator.class_indices

## Assigning Labels Image X with label y for Mesonet

## Display Prediction with 1 sample image

In [ ]:
X, y = generator.next()

#evaluating prediction
print(f"Predicted likelihood: {meso.predict(X)[0][0]:.4f}")
print(f'\nActual label: {int(y[0])}')
print(f"\nCorrect prediction: {round(meso.predict(X)[0][0]==y[0])}")

#display images
plt.imshow(np.squeeze(X));

## Lists for correctly classified and misclassified images

In [ ]:
correct_real = []
correct_real_pred = []

correct_deepfake = []
correct_deepfake_pred = []

misclassified_real = []
misclassified_real_pred = []

misclassified_deepfake = []
misclassified_deepfake_pred = []

## Generating Predictions on Validation set, storing in the lists

In [ ]:
for i in range(len(generator.labels)):
  #load next pic
  X,y = generator.next()

  if round(meso.predict(X)[0][0]==y[0] and y[0] ==1)
      correct_real.append(X)
      correct_real_pred.append(meso.predict(X)[0][0])
  elif round(meso.predict(X)[0][0]==y[0] and y[0] ==1):
      correct_deepfake.append(X)
      correct_deepfake_pred.append(meso.predict(X)[0][0])

  elif round(meso.predict(X)[0][0] != y[0] and y[0] ==1):
      misclassified_real.append(X)
      misclassified_real_pred.append(meso.predict(X)[0][0])

  else:
      misclassified_deepfake.append(X)
      misclassified_deepfake_pred.append(meso.predict(X)[0][0])

  if i % 1000 == 0:
    print(i, "predictions completed.")

  if i == len(generator.labels)-1:
    print("All", len(generator.labels), "predictions completed.")



## Plot Data

In [ ]:
def plotter(images, preds):
  fig = plt.figure(figsize = (16,9))
  subset = np.random(randint(0,len(images)-1,12))
  for i,j in enumerate(subset):
    fig.add_subplot(3,4,i+1)
    plt.imshow(np.squeeze(images[j]))
    plt.xlabel(f"Model confience: \n{preds[j]:.4f}")
    plt.tight_layout()
    ax = plt.gca()
    ax.axes.xaxis.set_ticks([])
    ax.axes.yaxis.set_ticks([])
  plt.show;
  return


##Show Plot

For correct real

In [ ]:
plotter(correct_real, correct_real_pred)

Misclassified real

In [ ]:
plotter(misclassified_real, misclassified_real_pred)

Correct deepfake

In [ ]:
plotter(correct_deepfake, correct_deepfake_pred)

Incorrect deepfake

In [ ]:
plotter(misclassified_deepfake, misclassified_deepfake_pred)